# "BC dates in Python"
> "Several approaches to expressing BC dates in Python"

- toc: false
- branch: master
- badges: false
- comments: true
- categories: [datascience, history, python]
- image: images/placeholder.png
- hide: false
- search_exclude: false

## The problem
A while ago, I was doing a small visualisation project on an ancient history topic, and as part of that, I needed to express dates and times around 400BC. Sounds simple enough, right? Python has a [datetime module](https://docs.python.org/3/library/datetime.html), just use that and we're golden.

Well, not so fast. The [MINYEAR](https://docs.python.org/3/library/datetime.html#datetime.MINYEAR) is 1AD, so we can't express BC dates like that. We'll need a different solution.

What are our requirements for a good solution? What functionality are we looking for?
- Expressing BC as well as AD dates
- Create from string and/or numeric parameters
- Print time
- Getters
- Add/subtract time span and getting time deltas
- Get time span delta
- Lightweight enough to use in pandas

In [1]:
#collapse-hide

# Imports
from datetime import datetime, timedelta

## 1. Data science libraries

### Numpy - datetime64 + timedelta64
Numpy's [`datetime64` and `timedelta64`](https://numpy.org/doc/stable/reference/arrays.datetime.html) extends Python's base `datetime`/`timedelta` from 32 to 64 bit and also adds some extra functionality.

In [2]:
#collapse-hide

import numpy as np

#### Numpy: Creating times BC/AD
Both classes support a range of +/-2.9e11 years with a second precision which should be enough for most purposes.

In [3]:
#collapse-show

# Example code for creating BC and AD times
ad_date_np = np.datetime64("2020-01-02T03:04:05")
print(ad_date_np)

bc_date_np = np.datetime64("-00400-01-02T03:04:05")
print(bc_date_np)

2020-01-02T03:04:05
-400-01-02T03:04:05


#### Numpy: Getters
Numpy doesn't provide extraction functions out of the box, but we can roll our own based on the string representation (inspired by [this stack overflow answer](https://stackoverflow.com/a/56260054)).

In [4]:
#collapse-show

def dt2cal(dt):
    """
    Convert datetime64 to a calendar array of year, month, day, hour, minute, seconds, microsecond.

    Parameters
    ----------
    dt : datetime64
        datetime

    Returns
    -------
    cal : int32 array (7)
        calendar array representing year, month, day, hour, minute, second, microsecond
    """

    # allocate output 
    out = np.empty(7, dtype="i4")
    
    # decompose calendar floors
    Y, M, D, h, m, s = [dt.astype(f"M8[{x}]") for x in "YMDhms"]
    out[0] = Y.astype(int) + 1970 # Gregorian year
    out[1] = (M - Y) + 1 # month
    out[2] = (D - M).astype(int) + 1 # day
    out[3] = (dt - D).astype("m8[h]").astype(int) # hour
    out[4] = (dt - h).astype("m8[m]").astype(int) # minute
    out[5] = (dt - m).astype("m8[s]").astype(int) # second
    out[6] = (dt - s).astype("m8[us]").astype(int) # microsecond
    
    return out

In [5]:
#collapse-show

cal = dt2cal(bc_date_np)
cal

array([-400,    1,    2,    3,    4,    5,    0], dtype=int32)

In [6]:
#collapse-hide

def dt2calarray(dt):
    """
    Convert array of datetime64 to a calendar array of year, month, day, hour,
    minute, seconds, microsecond with these quantites indexed on the last axis.

    Parameters
    ----------
    dt : datetime64 array (...)
        numpy.ndarray of datetimes of arbitrary shape

    Returns
    -------
    cal : uint32 array (..., 7)
        calendar array with last axis representing year, month, day, hour,
        minute, second, microsecond
    """

    # allocate output 
    out = np.empty(dt.shape + (7,), dtype="i4")
    # decompose calendar floors
    Y, M, D, h, m, s = [dt.astype(f"M8[{x}]") for x in "YMDhms"]
    out[..., 0] = Y.astype(int) + 1970 # Gregorian year
    out[..., 1] = (M - Y) + 1 # month
    out[..., 2] = (D - M).astype(int) + 1 # dat
    out[..., 3] = (dt - D).astype("m8[h]").astype(int) # hour
    out[..., 4] = (dt - h).astype("m8[m]").astype(int) # minute
    out[..., 5] = (dt - m).astype("m8[s]").astype(int) # second
    out[..., 6] = (dt - s).astype("m8[us]").astype(int) # microsecond
    return out

In [7]:
#collapse-show

dates_np = np.array([bc_date_np, ad_date_np], dtype='datetime64')
cal_arr = dt2calarray(dates_np)
cal_arr

array([[-400,    1,    2,    3,    4,    5,    0],
       [2020,    1,    2,    3,    4,    5,    0]], dtype=int32)

### Numpy: Time spans
`timedelta64` handles addition and subtraction as expected; the only thing worth calling out is the concept of time scale inherent in Numpy's time classes: To operate on two `timedelta64` objects, they both need to have the same time scale, i.e. both need to have the `[Y]`, `[d]`, etc format. You can easily convert these using `.astype(timedelta64[X])`.

This also holds for printing: The first print in the code below prints as seconds, which isn't very useful, but you can cast it to a better time scale such as years for better readability. Similarly, you need to specify the time scale when creating a time delta from scratch.

In [8]:
#collapse-show

delta_np = ad_date_np - bc_date_np
print(delta_np)
print(delta_np.astype("timedelta64[D]"))
print(delta_np.astype("timedelta64[Y]"))

new_delta_np = np.timedelta64(123, 'D')
print(new_delta_np)

print(delta_np - new_delta_np)

print(ad_date_np + new_delta_np)

76367836800 seconds
883887 days
2420 years
123 days
76357209600 seconds
2020-05-04T03:04:05


### Conclusion
Numpy is a solid choice if all you need is to cover a large range of times. Creating and dealing with dates and time spans is fairly straightforward, and the objects are lean as can be. As we've seen, we can compensate for the lack of easy extraction functions with our own workarounds, but it's not as efficient as if it was supported natively, and is prone to breaking as the language evolves.

### Pandas
[Pandas](https://pandas.pydata.org/) is the standard framework many data scientists use when dealing with large amounts of data. Its [`to_datetime`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) method is a great way to convert columns of dates from strings to Pandas [`Timestamps`](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timestamps-vs-time-spans) which wrap Numpy's `datetime64`. 

In [9]:
#collapse-hide

#Imports
import pandas as pd

In [10]:
#collapse-show

df = pd.DataFrame({'date': ['1970-1-2 03:04:05', '2020-6-7 08:09:10'],
                   'value': [2, 3]})
df['date'] = pd.to_datetime(df['date'], format="%Y-%d-%m %H:%M:%S")
df

,date,value
0,1970-02-01 03:04:05,2
1,2020-07-06 08:09:10,3


#### Pandas: Creating times BC/AD
Since Pandas internally uses `datetime64`, we should just be able to use `to_datetime` on both BC and AD dates and we're all good to go, right? Wrong. While Pandas does use 64 bits to represent time, it does so at a precision of nanoseconds which severly limits the available date range as discussed [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-timestamp-limits).

In [11]:
#collapse-show

# This will result in an error:
#date = pd.to_datetime('-400-01-02T03:04:05', format="%Y-%d-%m %H:%M:%S")

If you try to parse a value before 1677AD, you'll get this error:

`ValueError: time data '-400-01-02T03:04:05' does not match format '%Y-%d-%m %H:%M:%S' (match)`.

The error message is a bit misleading since the input format is just fine, the problem is that the date itself falls outside the supported range. So `to_datetime` is out.

What if we populated our data frames directly with `datetime64`?

In [12]:
#collapse-show

df = pd.DataFrame({'date': [np.datetime64('-400-01-02T03:04:05'), np.datetime64('2020-06-07T08:09:10')],
                   'value': [2, 3]})
df

,date,value
0,-400-01-02T03:04:05,2
1,2020-06-07T08:09:10,3


This works beautifully! Looks like we can get our BC dates into Pandas data frames after all.

So we can use `datetime64` directly, we just don't have the convenience of assembling `Timestamps` straight from string column(s). Here's a utility function that converts `datetime64`-compatible strings and puts them back into the column.

In [13]:
#collapse-show

def custom_to_datetime_from_date(df):
    df['date'] = df.apply(lambda row: np.datetime64(row['date']), axis=1)
    return df

In [14]:
#collapse-show

test_df = pd.DataFrame({'date': ['-400-01-02T03:04:05', '2020-06-07T08:09:10'],
                   'value': [2, 3]})
print(test_df)

test_df = custom_to_datetime_from_date(test_df)
test_df

                  date  value
0  -400-01-02T03:04:05      2
1  2020-06-07T08:09:10      3


,date,value
0,-400-01-02T03:04:05,2
1,2020-06-07T08:09:10,3


Another great use of `to_datetime` is its ability to assemble `Timestamps` from multiple colums, e.g. `year`, `month`, etc. Here's what a workaround for this could look like:

In [15]:
#collapse-show

helper_df = pd.DataFrame({'column_names': ['month', 'day', 'hour', 'minute', 'second', 'ms', 'ns'],
                          'date_args': ['M', 'D', 'h', 'm', 's', 'ms', 'ns'],
                          'offsets': [1, 1, 0, 0, 0, 0, 0]})

print(helper_df)

def row_to_date_from_time_columns(row):
    if not 'year' in row:
        return None
    
    date = np.datetime64(row['year'], 'Y')
    
    for idx, r in helper_df.iterrows():
        name = r['column_names']
        if name in row:
            arg = r['date_args']
            date += np.timedelta64(row[name], arg)
            
            offset = r['offsets']
            if offset != 0:
                date -= np.timedelta64(offset, arg)
    
    return date

def custom_to_datetime_from_time_columns(df, drop_source_columns=False):
    df['date'] = df.apply(lambda row: row_to_date_from_time_columns(row), axis=1)
    
    if drop_source_columns:
        df = df.drop(helper_df['column_names'], axis=1, errors='ignore')
    
    return df

  column_names date_args  offsets
0        month         M        1
1          day         D        1
2         hour         h        0
3       minute         m        0
4       second         s        0
5           ms        ms        0
6           ns        ns        0


In [16]:
#collapse-show

test_df = pd.DataFrame({'year': ['-400', '2020'],
                        'month': ['01', '02'],
                        'day': ['03', '04'],
                        'value': [2, 3]})
print(test_df)

test_df = custom_to_datetime_from_time_columns(test_df, True)
test_df

   year month day  value
0  -400    01  03      2
1  2020    02  04      3


,year,value,date
0,-400,2,-400-01-03
1,2020,3,2020-02-04


For efficiency's sake, I've moved some of the arrays outside the function. You should further adjust this by removing the columns you don't need, but this should be a good starting point.

And finally, a combined version that converts the `date` column if it exists, or assembles time-related columns if it doesn't:

In [17]:
#collapse-show

helper_df = pd.DataFrame({'column_names': ['month', 'day', 'hour', 'minute', 'second', 'ms', 'ns'],
                          'date_args': ['M', 'D', 'h', 'm', 's', 'ms', 'ns'],
                          'offsets': [1, 1, 0, 0, 0, 0, 0]})

def row_to_date(row):
    if 'date' in row:
        return np.datetime64(row['date'])
    
    if not 'year' in row:
        return None
    
    date = np.datetime64(row['year'], 'Y')
    
    for idx, r in helper_df.iterrows():
        name = r['column_names']
        if name in row:
            arg = r['date_args']
            date += np.timedelta64(row[name], arg)
            
            offset = r['offsets']
            if offset != 0:
                date -= np.timedelta64(offset, arg)
    
    return date

def custom_to_datetime(df, drop_source_columns=False):
    df['date'] = df.apply(lambda row: row_to_date(row), axis=1)
    
    if drop_source_columns:
        df = df.drop(helper_df['column_names'], axis=1, errors='ignore')
    
    return df

Great, so we've worked around that limitation in Pandas! However, this still doesn't solve the `Timestamp` limitation in `DatetimeIndex` and `PeriodIndex`.

Since we can create columns of `datetime64` from strings now, maybe we can use the origin parameter of `datetime64` and turn them into dates based on the Julian calender? This would mean that we can get a `datetime64` with origin 4713BC that counts days from that date. Sounds like a good solution? Can we trick `Timestamp` into accepting a valid `datetime64` and have it not modify it?

In [18]:
#collapse-show

julian_origin = np.datetime64('-4713-01-01T12:00:00')
test_input = np.datetime64('-400-01-02')

# This will error out with:
# OutOfBoundsDatetime: 1574925 is Out of Bounds for origin='julian'
#pd.to_datetime((test_input - julian).astype('timedelta64[D]').astype(int), unit='D', origin='julian')

Nope, doesn't work. There is simply no way to get `Timestamp` to extend its range, even if you pass in a Julian `datetime64`. Pandas takes that representation and converts it to `datetime64[ns]` which again only goes as far back as 1677AD.

Pandas gives you false hope by stating that you can provide a different origin to `to_datetime` like this: `pd.to_datetime([1, 2, 3], unit="D", origin=pd.Timestamp("1960-01-01"))`. However, for that to work, the origin `Timestamp` needs to be valid in the first place. So we'd need to create a BC `Timestamp` to use as an origin point to get BC `Timestamps`, which we can't do because BC `Timestamps` aren't valid. Great. I guess we just can't use `DateTimeIndex`.

#### Pandas: Getters
`Timestamp` and `DatetimeIndex` have a great selection of component getters (as described [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components). Sadly, we can't use any of them since we can't use `Timestamp`/`DatetimeIndex`...

The best we can do is use the getters defined [above](#numpy-getters) as a starting point and write per-component extractor utilities instead of returning the full array. I'll leave this as an exercise to the reader ;-)

### Pandas: Time spans
One would think that if you have a working column of `datetime64` objects and try to apply `timedelta64` operations to the whole column, it would work. However, Pandas helpfully attempts to convert the column to `Timestamps` before applying the operation, which of course fails since our data is outside the `Timestamp` limits.
Instead, we have to `apply` operations row-by-row, not using the column shorthand.

(Again, note that type casts are required to make the `datetime64` in second-format compatible with the `timedelta64` which is in year-format due to its construction.)

In [19]:
#collapse-show

test_df = pd.DataFrame({'date': ['-400-01-02T03:04:05', '2020-06-07T08:09:10'],
                   'value': [2, 3]})

test_df = custom_to_datetime(test_df)

sub_ts = np.timedelta64(100, 'Y')

# Pandas will attempt to convert the date column into a Timestamp and crashes as a result
#test_df['date'] -= sub_ts

test_df['date'] = df.apply(lambda row: row['date'] - sub_ts.astype('m8[s]'), axis=1)

test_df

,date,value
0,-500-01-01T21:04:05,2
1,1920-06-08T02:09:10,3


### Conclusion
You can make Pandas work with Numpy's 64-bit time classes, it just needs some workarounds and a careful approach to keep Pandas from accidentally attempting an automatic conversion. Pandas usually adds a lot of utility to extend Numpy's time handling, but since they're all tied into range-limited classes, we can't use most of them.

Some articles that discuss the advantages of Pandas' time handling can be found [here](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html) and [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html), just so you know what you're missing out on.

## 2. Scientific libraries
It looks like there aren't a whole lot of historians using Python, but who know who else uses BC dates? Astronomers! There appears to be quite a selection of scientific libraries that deal with dates outside the default Python date range.

### [Astropy](https://www.astropy.org/)
Astropy's [Time](https://docs.astropy.org/en/stable/api/astropy.time.Time.html) module does the job. It supports a wide variety of time scales, formats and precision that are handy in general, plus some highly astronomy specific functionality such as `earth_rotation_angle` and `light_travel_time`. The [documentation](https://docs.astropy.org/en/stable/time/) does a good job explaining how to create and work with `Time`.

This is an incredibly powerful library, and the documentation rocks. The `Time` object supports common operations such as creating, modifying and printing dates and time spans.

One downside is that it has a lot of functionality you likely won't need, so it looks intimidating and confusing at first glance. While it looks like this added functionality would also bloat the object, this isn't a problem in practice. In addition to taking a string as input, `Time` also takes an `ndarray` of strings, and can be interacted with like an array. This means that effectively, you only need to define your time format and such once for the `Time` object, and the individual times can then be accessed by index.

In [20]:
#collapse-hide

#Imports
from astropy.time import Time, TimeDelta

#### Astropy: Creating times BC/AD
One caveat is that we shouldn't use the standard [ISO/ISOT format](https://docs.astropy.org/en/stable/api/astropy.time.TimeISOT.html#astropy.time.TimeISOT), e.g. `2020-01-02T03:04:05`. The ISO format only works for AD dates! Instead, we should use the [FITS format](https://docs.astropy.org/en/stable/api/astropy.time.TimeFITS.html#astropy.time.TimeFITS), e.g. `-00400-01-02T03:04:05`. FITS is an extension to the ISO format that expands the year range to five digits, and supports BC dates. Be aware that for negative dates, you need to pad it out with leading zeroes, or you'll get a parsing exception!

In [21]:
#collapse-show

# Example code for creating BC and AD times
ad_date_astro = Time("2020-01-01T01:23:45", format='fits', scale='utc')
print(ad_date_astro)

bc_date_astro = Time("-00400-01-01T01:23:45", format='fits', scale='utc')
print(bc_date_astro)

dates_astro = Time(["-00400-01-01T01:23:45", "2020-01-01T01:23:45"], format='fits', scale='utc')
print(dates_astro)
print(dates_astro[0])

2020-01-01T01:23:45.000
-00400-01-01T01:23:45.000
['-00400-01-01T01:23:45.000' '+02020-01-01T01:23:45.000']
-00400-01-01T01:23:45.000


/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


The above warning seems to be benign and from well within Astropy, nothing to do for us here.

Side note: You can get your time data in a different format by using `.isot`, `.unix`. Note that you can print time in formats that wouldn't support parsing it!

In [22]:
#collapse-show

print(ad_date_astro.unix)
print(bc_date_astro.fits)
print(bc_date_astro.isot)

# This line would fail with a ValueError: "Input values did not match the format class isot"
#bc_date_2 = Time(bc_date.isot, format='isot', scale='utc')

1577841825.0
-00400-01-01T01:23:45.000
-400-01-01T01:23:45.000


#### Astropy: Getters
The `Time` class doesn't have an easy way to extract common attributes such as year, month or seconds. It does, however, support `strftime` and provides an implementation based on Python's [`time.strftime`](https://docs.python.org/3/library/time.html#time.strftime). This means we can define our own utility functions to extract the relevant components and parse them into integers for later use.

Sadly, this doesn't work for BC dates since `strftime` internally assumes dates in the `ISO` format for some reason.

In [23]:
#collapse-show

#Getters
def extract_year(t):
    return int(t.strftime('%Y'))

extract_year(ad_date_astro)

2020

There is always the option of writing our own function to extract time components from the source string, and I might revisit this section later on to do just that. We'd have to use the FITS representation and parse the date string ourselves which shouldn't be too involved.

In [24]:
#collapse-hide

#TODO: Write time component extraction function based on FITS string representation

### Astropy: Time spans
Astropy comes with a `TimeDelta` class that supports all common time span operations, such as add/subtract and new instance creation. The most common constructors are from seconds or from [`datetime.timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects).

In [25]:
#collapse-show

delta_astro = ad_date_astro - bc_date_astro
print(delta_astro.datetime)

new_delta_astro = TimeDelta(123456789, format='sec')
print(new_delta_astro.datetime)

new_delta_astro = TimeDelta(timedelta(days=123456), format='datetime')
print(new_delta_astro.datetime)

print((delta_astro - new_delta_astro).datetime)
print(ad_date_astro + new_delta_astro)

883887 days, 0:00:37
1428 days, 21:33:09
123456 days, 0:00:00
760431 days, 0:00:37
2358-01-05T01:23:45.000


/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/lib/python3.8/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


### Conclusion
Astropy's time module offers a lot more functionality than we need which makes its objects clunky and its API quite complex. This is offset by having the option to keep an array of datetimes in the `Time` object itself, same as with `TimeDelta`, which lets us manipulate them as we would with `ndarrays`. The downside is that this wouldn't work well with libraries such as Pandas.

### [Skyfield](https://rhodesmill.org/skyfield/time.html)
TODO: Overview

#### Skyfield: Creating times BC/AD


In [26]:
#collapse-hide

# TODO

#### Skyfield: Getters


In [27]:
#collapse-hide

# TODO

#### Skyfield: Time spans


In [28]:
#collapse-hide

# TODO

#### Conclusion
TODO

### [SpiceyPy](https://github.com/AndrewAnnex/SpiceyPy)

This is more of an honourable mention than anything else. SpiceyPy is a wrapper for the C-based [SPICE toolkit](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/info/intrdctn.html), and it shows in it's usability and documentation. For a taste, check out this [easy-to-follow tutorial](https://spiceypy.readthedocs.io/en/main/remote_sensing.html#overview) full of kernel installs and C-style code.

It does come with time utilities, but I couldn't even figure out how to set them up, let alone use them, so maybe steer clear.

## 3. Custom made date classes
The best custom-made date class I've found is this `FlexiDate`-inspired one [here](https://github.com/okfn/datautil/blob/master/datautil/date.py).

TODO: Overview

#### FlexiDate: Creating times BC/AD


In [29]:
#collapse-hide

# TODO

#### FlexiDate: Getters


In [30]:
#collapse-hide

# TODO

#### FlexiDate: Time spans


In [31]:
#collapse-hide

# TODO

#### Conclusion
TODO